# Resampling

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Data loading and splitting

In [12]:
Auto = load_data('Auto')
# int(Auto.shape[0] / 2) can replace with 196
Auto_train,Auto_valid = train_test_split(Auto, test_size = int(Auto.shape[0] / 2), random_state=0)


### Training the model with training data split

In [24]:
hp_mm = MS(['horsepower'])
X_train =hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model =sm.OLS(y_train, X_train)
results = model.fit()

### Evaluate the model matrix for this model created using the validation data set

In [25]:
X_valid = hp_mm.transform(Auto_valid)
y_valid =Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)


23.61661706966988

#### Hence our estimate for the validation MSE of the linear regression fit is: 23.62.

### A function evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set

In [33]:
def evalMSE(terms ,response , train , test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])